In [7]:
import numpy as np
path = "gdb10.dat"
# path="D:\PythonProject\CARP\sample.dat"
with open(path, "r") as f:
    content = f.readlines()
# print (content)
NAME = content[0].split(":")[1]
VERTICES_NUMBER = (int)(content[1].split(":")[1])
DEPOT = (int)(content[2].split(":")[1])
REQUIRED_EDGES = (int)(content[3].split(":")[1])
NON_REQUIRED_EDGES = (int)(content[4].split(":")[1])
VEHICLES = (int)(content[5].split(":")[1])
CAPACITY = (int)(content[6].split(":")[1])
RequiredEdgeCost = (int)(content[7].split(":")[1])
EdgeNumber = REQUIRED_EDGES+NON_REQUIRED_EDGES


class edge:
    def __init__(self, node1, node2, cost, demand):
        self.node1 = node1
        self.node2 = node2
        self.cost = cost
        self.demand = demand


# class node:
#     visited
#     children
edges = []
nodeToEdge = {}
for i in range(EdgeNumber):
    content[9+i] = content[9+i].split()
    Edge = edge((int)(content[9+i][0])-1, (int)(content[9+i][1]) -
                1, (int)(content[9+i][2]), (int)(content[9+i][3]))
    nodeToEdge[(Edge.node1, Edge.node2)] = Edge
    nodeToEdge[(Edge.node2, Edge.node1)] = Edge
    edges.append(Edge)
for i in range(EdgeNumber):
    print(edges[i].node1+1, end=" ")
    print(edges[i].node2+1, end=" ")
    print(edges[i].cost, end=" ")
    print(edges[i].demand)
# print(nodeToEdge.get((0,1)))
# print(nodeToEdge.get((0,1)).node1)


1 8 9 1
1 9 4 2
1 10 13 1
1 11 12 2
2 1 11 1
2 3 15 1
2 4 18 2
2 5 8 2
3 4 8 2
3 8 18 2
3 9 6 1
4 1 7 1
4 6 10 2
4 11 17 1
5 1 9 2
5 6 15 1
6 11 3 2
7 2 6 1
7 4 11 2
7 5 5 1
8 9 14 2
8 12 5 1
9 10 19 1
10 12 2 2
11 12 7 1


In [8]:
MAX = 1<<32-1
distance_matrix=np.ones((VERTICES_NUMBER,VERTICES_NUMBER),dtype=int)*MAX
for Edge in edges:
    node1=Edge.node1
    node2=Edge.node2
    distance_matrix[node1][node2]=Edge.cost
    distance_matrix[node2][node1]=Edge.cost

def dijkstra(matrix, start_node):
    matrix_length = len(matrix)  # 矩阵一维数组的长度，即节点的个数

    used_node = [False] * matrix_length  # 访问过的节点数组

    distance = [MAX] * matrix_length  # 最短路径距离数组

    distance[start_node] = 0  # 初始化，将起始节点的最短路径修改成0

    # 将访问节点中未访问的个数作为循环值，其实也可以用个点长度代替。
    while used_node.count(False):
        min_value = MAX
        min_value_index = -1

        # 在最短路径节点中找到最小值，已经访问过的不在参与循环。
        # 得到最小值下标，每循环一次肯定有一个最小值
        for index in range(matrix_length):
            if not used_node[index] and distance[index] <= min_value:
                min_value = distance[index]
                min_value_index = index

        # 将访问节点数组对应的值修改成True，标志其已经访问过了
        used_node[min_value_index] = True

        # 更新distance数组。
        # 以B点为例：distance[x] 起始点达到B点的距离。
        # distance[min_value_index] + matrix[min_value_index][index] 是起始点经过某点达到B点的距离，比较两个值，取较小的那个。
        for index in range(matrix_length):
            distance[index] = min(distance[index], distance[min_value_index] + matrix[min_value_index][index])

    return distance
# dijkstra(distance_matrix,0)
Shortest_Distance=[]
for i in range(VERTICES_NUMBER):
    Shortest_Distance.append(dijkstra(distance_matrix,i))

In [9]:
#Path Scanning
# print(VERTICES_NUMBER)
# print(Shortest_Distance)
#Rule:
# 1) maximize the distance from the task to the depot;
# 2) minimize the distance from the task to the depot;
# 3) maximize the term dem(t)/sc(t), where dem(t) and sc(t) are demand and serving cost of task t, respectively;
# 4) minimize the term dem(t)/sc(t);
# 5) use rule 1) if the vehicle is less than half- full, otherwise use rule 2)
#better returns whether u1 is better than u2
def better(u1,u2,load,rule):
    node1=u1.node1
    node2=u2.node1
    distance1=Shortest_Distance[node1][DEPOT-1]
    distance2=Shortest_Distance[node2][DEPOT-1]
    if rule==1:
        if distance1 >=distance2:
            return True
        else:
            return False
    elif rule==2:
        if distance1 <=distance2:
            return True
        else:
            return False
    elif rule==3:
        if u1.demand/u1.cost >=u2.demand/u2.cost:
            return True
        else:
            return False
    elif rule==4:
         if u1.demand/u1.cost <=u2.demand/u2.cost:
            return True
         else:
            return False
    else:
        if load<=CAPACITY/2:
            return True
        else:
            return False



#需求边集free
def path_scanning(rule):
    free=[]
    k=0
    for u in edges:
        if u.demand!=0:
            free.append((u.node1,u.node2))
            free.append((u.node2,u.node1))
    routines=[]
    costs=[]
    while len(free)!=0 and k<VEHICLES:
        routine=[]
        k=k+1
        load=0
        cost=0
        i=DEPOT-1
        d=0
        # d=MAX
        while len(free)!=0 and d!=MAX :
            d=MAX
            for item in free :
                u=nodeToEdge.get(item)
                if load+u.demand<=CAPACITY:
                    distance=Shortest_Distance[i][item[0]]
                    if distance<d:
                        d=distance
                        chosen_arc=item
                    elif distance==d and better(u,nodeToEdge.get(chosen_arc),load,rule):
                        chosen_arc=item

            if d!=MAX:
                chosen_edge=nodeToEdge.get(chosen_arc)
                routine.append(chosen_arc)
                free.remove(chosen_arc)
                free.remove((chosen_arc[1],chosen_arc[0]))
                load=load+chosen_edge.demand
                cost=cost+d+chosen_edge.cost
                i=chosen_arc[1]
        cost=cost+Shortest_Distance[i][DEPOT-1]
        routines.append(routine)
        costs.append(cost)
    if len(free)!=0:
        return -1
    return routines, sum(costs)

# print(routines)
# print(sum(costs))
Solution, Cost = [], MAX
for i in range(5):
    solution,cost=path_scanning(i)
    if cost<Cost:
        Solution=solution
        Cost=cost
print("s",end=" ")
for i in range(len(Solution)):
    print(0,end=",")
    for j in range(len(Solution[i])):
        Solution[i][j]=(Solution[i][j][0]+1,Solution[i][j][1]+1)
        print(Solution[i][j],end=",")
    if(i==len(Solution)-1 and j==len(Solution[i])-1):
        print(0,end="")
    else:
        print(0,end=",")
print()
print("q",end=" ")
print(Cost)
# for i in range(EdgeNumber):
#     print(edges[i].node1,end=" ")
#     print(edges[i].node2,end=" ")
#     print(edges[i].cost,end=" ")
#     print(edges[i].demand)

# print(NAME)
# print(VERTICES_NUMBER)
# print(DEPOT)
# print(REQUIRED_EDGES)
# print(NON_REQUIRED_EDGES)
# print(VEHICLES)
# print(CAPACITY)
# print(RequiredEdgeCost)

# content[1]

s 0,(1, 2),(2, 7),(7, 5),(5, 2),(2, 4),(4, 7),(5, 6),0,0,(1, 5),(2, 3),(3, 9),(9, 8),(8, 3),(3, 4),0,0,(1, 4),(4, 11),(11, 6),(6, 4),(1, 11),(11, 12),(12, 8),0,0,(1, 10),(10, 12),(10, 9),(9, 1),(1, 8),0
q 314


In [10]:
def fitness(routines):
    cost = 0
    for i in range(len(routines)):
        route_cost = 0
        start = DEPOT-1
        for j in range(len(routines[i])):
            print(route_cost)

            begin = routines[i][j][0]-1
            end = routines[i][j][1]-1
            print(Shortest_Distance[start][begin],nodeToEdge.get((begin, end)).cost)
            route_cost += Shortest_Distance[start][begin] + \
                nodeToEdge.get((begin, end)).cost
            start = end
        route_cost += Shortest_Distance[end][DEPOT-1]

        print(route_cost)
        print("-" * 50)
        cost += route_cost
    return cost


road = [[(1, 8), (8, 3), (3, 4), (4, 11), (11, 12), (10, 1)],
        [(1, 11), (11, 6), (6, 4), (4, 1), (9, 1)],
        [(1, 2), (2, 4), (4, 7), (7, 2), (2, 5), (5, 1)],
        [(6, 5), (5, 7), (2, 3), (3, 9), (9, 8), (8, 12), (12, 10), (10, 9)]]
fitness(road)


0
0 9
9
0 18
27
0 8
35
0 17
52
0 7
59
2 13
74
--------------------------------------------------
0
0 12
12
0 3
15
0 10
25
0 7
32
4 4
40
--------------------------------------------------
0
0 11
11
0 18
29
0 11
40
0 6
46
0 8
54
0 9
63
--------------------------------------------------
0
15 15
30
0 5
35
6 15
56
0 6
62
0 14
76
0 5
81
0 2
83
0 19
106
--------------------------------------------------


283

In [11]:
Shortest_Distance

[[0, 11, 10, 7, 9, 15, 14, 9, 4, 13, 12, 14],
 [11, 0, 15, 17, 8, 23, 6, 20, 15, 24, 23, 25],
 [10, 15, 0, 8, 19, 18, 19, 18, 6, 23, 21, 23],
 [7, 17, 8, 0, 16, 10, 11, 16, 11, 20, 13, 20],
 [9, 8, 19, 16, 0, 15, 5, 18, 13, 22, 18, 23],
 [15, 23, 18, 10, 15, 0, 20, 15, 19, 12, 3, 10],
 [14, 6, 19, 11, 5, 20, 0, 23, 18, 27, 23, 28],
 [9, 20, 18, 16, 18, 15, 23, 0, 13, 7, 12, 5],
 [4, 15, 6, 11, 13, 19, 18, 13, 0, 17, 16, 18],
 [13, 24, 23, 20, 22, 12, 27, 7, 17, 0, 9, 2],
 [12, 23, 21, 13, 18, 3, 23, 12, 16, 9, 0, 7],
 [14, 25, 23, 20, 23, 10, 28, 5, 18, 2, 7, 0]]